In [3]:
import pandas as pd
import numpy as np
import itertools as it
import os,re
from collections import defaultdict

In [8]:
import sys
sys.path.append("/cluster/bh0085/mybio")
import ipnb_utils

#dependencies from elsewhere in the C libraries
#from c_create_dictionaries import list_umis
from _config import DATA_DIR, OUT_DIR

inp_dir = os.path.join(OUT_DIR, "b_demultiplex")


In [46]:
split = "1"
exp = exps[0]
f_infos = dict([[fn,re.compile("(?P<exp>[^/]*)/R(?P<rnum>[12])_(?P<split>\d+).fa").search(fn).groupdict()] 
                 for fn in fa_found])

print split,  exp
f1 = [k for k,v in f_infos.items() if v["rnum"] == "1" and v["split"]==split and v["exp"]==exp][0] 
f2 = [k for k,v in f_infos.items() if v["rnum"] == "2" and v["split"]==split and v["exp"]==exp][0] 


nl = 0
nacc = 0
nnreject = 0
umis = []
t_umis = []
with open(f1) as f1open:
  with open(f2) as f2open:
    i = -1
    while 1:
        i+=1
        try:
            l1 = f1open.next()
            l2 = f2open.next()
        except StopIteration:
            break

        if i %2 == 0: continue
        if "N" in l2[:16]:
            nnreject +=1
            continue
        umi = l2[:15].strip()
        umis.append(umi)          
        t_umi = l1[:10].strip()
        t_umis.append(t_umi)
        nacc+=1


1 HCT116_WT_BR1


In [51]:
umis[0]

'GTCATGCAGACTTGC'

In [13]:

fa_found = []
def wfun(arg,dirname,fnames):
    for f in fnames:
        if f[-2:]=="fa":
            fa_found.append(os.path.join(dirname,f))
os.path.walk("../out/b_demultiplex/",wfun,None)
  

def list_umis(split="0",exp="HCT116_Gemcitabine2uM_BR2"):
    f_infos = dict([[fn,re.compile("(?P<exp>[^/]*)/R(?P<rnum>[12])_(?P<split>\d+).fa").search(fn).groupdict()] 
                     for fn in fa_found])
    f1 = [k for k,v in f_infos.items() if v["rnum"] == "1" and v["split"]==split and v["exp"]==exp][0] 
    f2 = [k for k,v in f_infos.items() if v["rnum"] == "2" and v["split"]==split and v["exp"]==exp][0] 


    nl = 0
    nacc = 0
    nnreject = 0
    umis = []
    t_umis = []
    with open(f1) as f1open:
      with open(f2) as f2open:
        i = -1
        while 1:
            i+=1
            try:
                l1 = f1open.next()
                l2 = f2open.next()
            except StopIteration:
                break

            if i %2 == 0: continue
            if "N" in l2[:16]:
                nnreject +=1
                continue
            umi = l2[:15].strip()
            umis.append(umi)          
            t_umi = l1[:10].strip()
            t_umis.append(t_umi)
            nacc+=1
    return umis, t_umis, {"nacc":nacc, "nnreject":nnreject}

In [55]:
sum([len(e) for v in all_bcs2umis.values() for e in v.values()])
#40m for one experiment
#499m for all?

499603549

In [31]:
print len(umis)
print len(t_umis)

11455850
3085892


In [19]:
all_bcs2umis = dict()

exps = os.listdir(inp_dir)
total_count = 0 
for exp in exps:
    print exp
    all_bcs2umis[exp] = defaultdict(list)
    for split in [str(e) for e in range(15)]:
        print split
        

        bcs,umis,stats = list_umis(split = split, exp =exp)
        for i,bc in enumerate(bcs):
            all_bcs2umis[exp][bc].append(umis[i])

HCT116_WT_BR1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
HCT116_WT_BR2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
HCT116_Gemcitabine2uM_BR1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
HCT116_Gemcitabine2uM_BR2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
DLD1_WT_BR1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
DLD1_WT_BR2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
U2OS_WT_BR1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
U2OS_WT_BR2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
other
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [27]:
len(all_bcs2umis)

9

In [30]:
len(all_bcs2umis)

9

In [33]:
len(all_bcs2umis.values()[0])

7961056

In [29]:
for k,v in all_bcs2umis.iteritems():
    print k
    with open("../data/all_{}_bcs2umis.json".format(k), "w") as f:
        f.write(json.dumps(v))
    

DLD1_WT_BR2
DLD1_WT_BR1
HCT116_WT_BR2
HCT116_WT_BR1
U2OS_WT_BR1
other
U2OS_WT_BR2
HCT116_Gemcitabine2uM_BR2
HCT116_Gemcitabine2uM_BR1


In [ ]:
import json
with open("../data/all_bcs2umis.json", "w") as f:
    f.write(json.dumps(all_bcs2umis))

In [ ]:
print "HI"

# PLOTTING